In [73]:
import pandas as pd
import numpy as np
from scipy import stats 
from datetime import datetime

## Read in the raw taxi data.  Since the data is quite large we need to load it in chunks

In [2]:
%%time
#read the original taxi trip data csv file into chunklist
chunklist = []
for chunk in pd.read_csv('2016_Yellow_Taxi_Trip_Data.csv', chunksize=100000):
     chunklist.append(chunk)

CPU times: user 4min 9s, sys: 50.2 s, total: 4min 59s
Wall time: 5min 6s


In [3]:
%%time
#combine the chunks into a dataframe
df_taxi = pd.concat(chunklist)

CPU times: user 50.7 s, sys: 3min 16s, total: 4min 7s
Wall time: 9min 43s


In [4]:
df_taxi.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,...,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PULocationID,DOLocationID
0,1,02/22/2016 06:14:21 PM,02/22/2016 06:27:21 PM,1,4.4,-74.012749,40.702610,1,N,-73.987106,...,1,15.0,1.0,0.5,1.00,0.0,0.3,17.80,NaN,NaN
1,1,02/10/2016 05:51:14 PM,02/10/2016 05:58:34 PM,1,1.1,-73.986183,40.757881,1,N,-73.981720,...,1,6.5,1.0,0.5,1.65,0.0,0.3,9.95,NaN,NaN
2,1,02/13/2016 02:56:14 PM,02/13/2016 03:13:35 PM,1,6.6,-73.981140,40.782425,1,N,-73.930641,...,2,22.0,0.0,0.5,0.00,0.0,0.3,22.80,NaN,NaN
3,1,02/15/2016 01:29:39 PM,02/15/2016 01:45:53 PM,2,2.2,-73.998619,40.764027,1,N,-73.981750,...,2,12.0,0.0,0.5,0.00,0.0,0.3,12.80,NaN,NaN
4,1,02/27/2016 11:59:52 PM,02/28/2016 12:04:18 AM,1,1.2,-73.959244,40.763435,1,N,-73.946564,...,2,6.0,0.5,0.5,0.00,0.0,0.3,7.30,NaN,NaN


## Keep only the columns that we need (pickup, dropoff, and trip_distance)

In [5]:
df_taxi.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RatecodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'PULocationID',
       'DOLocationID'],
      dtype='object')

In [6]:
#drop the columns that we don't need to keep
df_taxi = df_taxi.drop(columns=['VendorID', 'passenger_count', 'RatecodeID', 'store_and_fwd_flag', 'payment_type',
                               'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 
                                'improvement_surcharge', 'total_amount', 'pickup_longitude', 'pickup_latitude', 
                                'dropoff_longitude', 'dropoff_latitude', 'PULocationID', 'DOLocationID'])

In [7]:
df_taxi.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance
0,02/22/2016 06:14:21 PM,02/22/2016 06:27:21 PM,4.4
1,02/10/2016 05:51:14 PM,02/10/2016 05:58:34 PM,1.1
2,02/13/2016 02:56:14 PM,02/13/2016 03:13:35 PM,6.6
3,02/15/2016 01:29:39 PM,02/15/2016 01:45:53 PM,2.2
4,02/27/2016 11:59:52 PM,02/28/2016 12:04:18 AM,1.2


### Rename the datetime columns 

In [8]:
df_taxi = df_taxi.rename(columns = {'tpep_pickup_datetime': 'pickup', 'tpep_dropoff_datetime':'dropoff'})

In [11]:
df_taxi.head()

,pickup,dropoff,trip_distance
0,02/22/2016 06:14:21 PM,02/22/2016 06:27:21 PM,4.4
1,02/10/2016 05:51:14 PM,02/10/2016 05:58:34 PM,1.1
2,02/13/2016 02:56:14 PM,02/13/2016 03:13:35 PM,6.6
3,02/15/2016 01:29:39 PM,02/15/2016 01:45:53 PM,2.2
4,02/27/2016 11:59:52 PM,02/28/2016 12:04:18 AM,1.2


## Convert the pickup and dropoff columns to datetime objects

In [23]:
%%time
df_taxi['pickup'] = pd.to_datetime(df_taxi['pickup'], infer_datetime_format = True)
df_taxi['dropoff'] = pd.to_datetime(df_taxi['dropoff'], infer_datetime_format = True)

CPU times: user 4h 38min 52s, sys: 2min 32s, total: 4h 41min 24s
Wall time: 4h 44min 40s


## Create a column trip_time from the dropoff and pickup datetime objects

In [25]:
df_taxi['trip_time'] = df_taxi.dropoff - df_taxi.pickup

In [26]:
df_taxi.head()

,pickup,dropoff,trip_distance,trip_time
0,2016-02-22 18:14:21,2016-02-22 18:27:21,4.4,00:13:00
1,2016-02-10 17:51:14,2016-02-10 17:58:34,1.1,00:07:20
2,2016-02-13 14:56:14,2016-02-13 15:13:35,6.6,00:17:21
3,2016-02-15 13:29:39,2016-02-15 13:45:53,2.2,00:16:14
4,2016-02-27 23:59:52,2016-02-28 00:04:18,1.2,00:04:26


## Create a speed column using the trip_distance and trip_time

In [28]:
df_taxi.loc[0,'trip_time'].total_seconds()

780.0

In [30]:
df_taxi['speed'] = df_taxi['trip_distance'] / (df_taxi['trip_time'].dt.total_seconds() / 3600)

In [31]:
df_taxi.head()

,pickup,dropoff,trip_distance,trip_time,speed
0,2016-02-22 18:14:21,2016-02-22 18:27:21,4.4,00:13:00,20.307692
1,2016-02-10 17:51:14,2016-02-10 17:58:34,1.1,00:07:20,9.000000
2,2016-02-13 14:56:14,2016-02-13 15:13:35,6.6,00:17:21,22.824207
3,2016-02-15 13:29:39,2016-02-15 13:45:53,2.2,00:16:14,8.131417
4,2016-02-27 23:59:52,2016-02-28 00:04:18,1.2,00:04:26,16.240602


In [32]:
df_taxi.speed.describe()

count    6.094906e+07
mean              inf
std               NaN
min     -3.698819e+08
25%      7.309645e+00
50%      1.005587e+01
75%      1.382199e+01
max               inf
Name: speed, dtype: float64

## Drop the rows where the speed is greater than 60, and less than 1.  Intuitively these limits seems reasonable 60 mph is very fast and 1 is very slow.

In [33]:
#drop rows where speed > 60
df_taxi = df_taxi.drop(df_taxi[df_taxi.speed > 60].index)

In [34]:
#drop rows where speed < 0
df_taxi = df_taxi.drop(df_taxi[df_taxi.speed < 1].index)

In [35]:
df_taxi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60592131 entries, 0 to 61084517
Data columns (total 5 columns):
pickup           datetime64[ns]
dropoff          datetime64[ns]
trip_distance    float64
trip_time        timedelta64[ns]
speed            float64
dtypes: datetime64[ns](2), float64(2), timedelta64[ns](1)
memory usage: 2.7 GB


In [37]:
df_taxi.speed.describe()

count    6.045667e+07
mean     1.145266e+01
std      6.160937e+00
min      1.000000e+00
25%      7.357664e+00
50%      1.008237e+01
75%      1.382488e+01
max      6.000000e+01
Name: speed, dtype: float64

## Checking the trip_distance column, we can see that there are at least some of the trips with a distance of 0.  These make no sense as well so let's get rid of them.

In [60]:
df_taxi.trip_distance.describe()

count    6.059213e+07
mean     2.920450e+00
std      3.615711e+00
min      0.000000e+00
25%      1.000000e+00
50%      1.700000e+00
75%      3.100000e+00
max      5.245000e+02
Name: trip_distance, dtype: float64

In [61]:
df_taxi = df_taxi.drop(df_taxi[df_taxi.trip_distance == 0].index)

In [62]:
df_taxi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60456674 entries, 0 to 61084517
Data columns (total 5 columns):
pickup           datetime64[ns]
dropoff          datetime64[ns]
trip_distance    float64
trip_time        timedelta64[ns]
speed            float64
dtypes: datetime64[ns](2), float64(2), timedelta64[ns](1)
memory usage: 2.7 GB


In [53]:
df_taxi_grouped = df_taxi.groupby([df_taxi.pickup.dt.month, df_taxi.pickup.dt.day])

## Using our list of days from the weather data, we will keep only the days that we want

In [67]:
days = [(11,29), (2,24), (5,3), (10,27), (3,4), (2,5), (3,21), (12,17), (7,2), (9,12)]

In [68]:
templist = []
for i in days:
    templist.append(df_taxi_grouped.get_group(i))

In [69]:
df_taxi_days = pd.concat(templist)

In [92]:
df_taxi_days.head()

,pickup,dropoff,trip_distance,trip_time,speed
32554703,2016-11-29 09:08:33,2016-11-29 09:13:15,0.5,00:04:42,6.382979
32554807,2016-11-29 12:22:40,2016-11-29 12:39:29,1.4,00:16:49,4.995045
32554904,2016-11-29 13:32:14,2016-11-29 13:52:12,1.5,00:19:58,4.507513
32555166,2016-11-29 07:13:08,2016-11-29 07:39:40,12.0,00:26:32,27.135678
32555652,2016-11-29 22:51:20,2016-11-29 23:08:06,4.1,00:16:46,14.671968


In [121]:
%%time
#Sort the dataframe by the pickup datetime
df_taxi_days.sort_values(by = ['pickup'], ascending = True, inplace = True)

CPU times: user 753 ms, sys: 543 ms, total: 1.3 s
Wall time: 1.33 s


In [124]:
df_taxi_days.reset_index(drop = True, inplace = True)

In [125]:
df_taxi_days.head()

,pickup,dropoff,trip_distance,trip_time,speed
0,2016-02-05 00:00:00,2016-02-05 00:08:02,1.5,00:08:02,11.203320
1,2016-02-05 00:00:01,2016-02-05 00:14:36,3.0,00:14:35,12.342857
2,2016-02-05 00:00:02,2016-02-05 00:21:48,4.0,00:21:46,11.026034
3,2016-02-05 00:00:02,2016-02-05 00:02:20,0.9,00:02:18,23.478261
4,2016-02-05 00:00:03,2016-02-05 00:05:17,0.6,00:05:14,6.878981


In [126]:
df_taxi_days.tail()

,pickup,dropoff,trip_distance,trip_time,speed
1688275,2016-12-17 23:59:58,2016-12-18 00:30:51,4.4,00:30:53,8.548300
1688276,2016-12-17 23:59:58,2016-12-18 00:22:15,10.2,00:22:17,27.464473
1688277,2016-12-17 23:59:58,2016-12-18 00:14:03,1.4,00:14:05,5.964497
1688278,2016-12-17 23:59:59,2016-12-18 00:13:56,2.4,00:13:57,10.322581
1688279,2016-12-17 23:59:59,2016-12-18 00:06:58,1.6,00:06:59,13.747017


In [127]:
%%time
#write the dataframe to a csv file
df_taxi_days.to_csv('taxi_edit.csv', index = False)

CPU times: user 43.3 s, sys: 1.15 s, total: 44.4 s
Wall time: 45.5 s
